In [3]:
import tabula
import pdfreader
from pdfreader import PDFDocument, SimplePDFViewer
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

import warnings
warnings.filterwarnings("ignore") 

ModuleNotFoundError: No module named 'tabula'

In [4]:
# Carregando o arquivo PDF
lista_tabelas = tabula.read_pdf("Relat_3.pdf", pages = 'all')

NameError: name 'tabula' is not defined

In [3]:
# Analise da quantas páginas que foram carregadas
print(len(lista_tabelas))

6


# Criação e preenchimento das TABELAS

In [4]:
# Obtendo somente a planilha 1 e transcrevendo os valores vazios com "0 - \r"
Grade_M = lista_tabelas[0].fillna(value="0 - \r")

In [5]:
# Obtendo somente a planilha 2 e transcrevendo os valores vazios com "0 - \r"
Grade_V = lista_tabelas[1].fillna(value="0 - \r")

In [6]:
# 0 - \r" serão usados para divisões das colunas futuramente
Grade_M = Grade_M.drop('Unnamed: 0', axis = 1)
Grade_V = Grade_V.drop('Unnamed: 0', axis = 1)
Grade_M = Grade_M.drop('Hr./Aula', axis = 1)
Grade_V = Grade_V.drop('Hr./Aula', axis = 1)

In [7]:
# Removendo as colunas sobresalentes das planilhas
Grade_M.columns = ["Horario",1,2,3,4,5]
Grade_V.columns = ["Horario",1,2,3,4,5]

# TRATAMENTO DAS COLUNAS

In [8]:
for h in range (0,5):
    Grade_M.iat[h,0] = Grade_M.iat[h,0][:5] + "\n" + Grade_M.iat[h,0][6:]

In [9]:
for h in range (0,5):
    Grade_V.iat[h,0] = Grade_V.iat[h,0][:5] + "\n" + Grade_V.iat[h,0][6:]

### COLETA DE DADOS DAS DICIPLINAS

In [10]:
# Matutino
Cod_Disc_M = [[],[],[],[],[]]
for i in range(1,6):
    Cod_Disc_M[i-1] = Grade_M[i].str.split(" - ", expand = True)[0]


In [11]:
# Vespertino
Cod_Disc_V = [[],[],[],[],[]]
for i in range(1,6):
    Cod_Disc_V[i-1] = Grade_V[i].str.split(" - ", expand = True)[0]


### COLETA DE DADOS DAS SALAS

In [12]:
# Matutino
Cod_Sala_M = [[],[],[],[],[]]
for i in range(1,6):
    Cod_Sala_M[i-1] = Grade_M[i].str.split("\r", expand = True)[1]


In [13]:
# Vespertino
Cod_Sala_V = [[],[],[],[],[]]
for i in range(1,6):
    Cod_Sala_V[i-1] = Grade_V[i].str.split("\r", expand = True)[1]


*FIM DO TRATAMENTO DE COLUNAS*

# Coleta *(Lista)* das diciplinas

In [14]:
#Localizando a tabela com as diciplinas (por padrão a ultima planilha)
diciplinas = lista_tabelas[len(lista_tabelas)-1]

In [15]:
coluna_diciplina = diciplinas.columns[2]

In [16]:
#Preenchendo os valores NULOS (NaN) da coluna de codicos das diciplinas por um valor (qualquer n nulo)
diciplinas[diciplinas.columns[0]] = diciplinas[diciplinas.columns[0]].fillna(value=int(1))

# Tratamento e associação das tabelas com a lista de diciplinas

In [17]:
dias = ["Segunda","Terça","Quarta","Quinta","Sexta"]

In [18]:
Grade_M.columns = ['Horário',]+dias
Grade_V.columns = ['Horário',]+dias

In [19]:
# Associando os codicos da grade dos dias das semanas com os codicos da tabela de diciplinas
for h in range(0,5):
    for j in range(0,5):
        for x in range(0,len(diciplinas)):
            # Caso o dia esteja sem aulas o horario permanece vazio
            if int(Cod_Disc_M[h][j]) == 0:
                Grade_M[dias[h]][j] = " "
                break
            else:
                # Reescrevimento da grade com os nomes das diciplinas
                if str(diciplinas.iat[x, 0]) == str(Cod_Disc_M[h][j]):
                    Grade_M[dias[h]][j] = diciplinas.iat[x, 2][:17] + "\n" + diciplinas.iat[x, 2][17:] + " - " + Cod_Sala_M[h][j]
                    break

In [20]:
# Associando os codicos da grade dos dias das semanas com os codicos da tabela de diciplinas
for h in range(0,5):
    for j in range(0,5):
        for x in range(0,len(diciplinas)):
            # Caso o dia esteja sem aulas o horario permanece vazio
            if int(Cod_Disc_V[h][j]) == 0:
                Grade_V[dias[h]][j] = " "
                break
            else:
                # Reescrevimento da grade com os nomes das diciplinas
                if str(diciplinas.iat[x, 0]) == str(Cod_Disc_V[h][j]):
                    Grade_V[dias[h]][j] = diciplinas.iat[x, 2][:17] + "\n" + diciplinas.iat[x, 2][17:] + " - " + Cod_Sala_V[h][j]
                    break

In [21]:
titulo_pdf = []
for i in Grade_M.columns:
    titulo_pdf += i,


In [22]:
teste = [titulo_pdf]

In [23]:
for j in range(0,len(titulo_pdf)-1):
    p0 = []
    for i in titulo_pdf:
        p0 += (Grade_M[i][j]),
    teste += p0,

In [24]:
for j in range(0,len(titulo_pdf)-1):
    p1 = []
    for i in titulo_pdf:
        p1 += (Grade_V[i][j]),
    teste += p1,

In [25]:
def create_pdf(file_name):
    
    page_size = landscape(letter)
    
    # Cria um documento
    doc = SimpleDocTemplate(file_name, pagesize=page_size)
    
    # Lista de elementos a serem adicionados ao documento
    elements = []

    # Define os dados da tabela
    horario_pdf = [teste]
    
    # Cria a tabela
    def_tab = 140
    table_dias = Table(teste, colWidths = [48, def_tab, def_tab, def_tab, def_tab, def_tab])
    
    # Define o estilo da tabela
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTSIZE', (1, 1), (-1, -1), 8),
        ('FONTSIZE', (0, 1), (0, -1), 9),
    ])
    table_dias.setStyle(style)
    
    # Adiciona a tabela à lista de elementos
    #elements.append(table)
    elements.append(table_dias)
    
    # Constrói o documento
    doc.build(elements)

if __name__ == "__main__":
    create_pdf("exemplo.pdf")
